step1_test_cwa_api

In [5]:
import requests
import json

API_KEY = "CWA-1F2E0711-4AD0-4F44-9344-E3799E4D8950"  # 換成你的 key
API_URL = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0002-001"

def debug_cwa_records():
    params = {
        "Authorization": API_KEY
    }

    print("🚀 呼叫中央氣象署 API ...")
    resp = requests.get(API_URL, params=params, timeout=15)
    print("HTTP 狀態碼：", resp.status_code)

    if resp.status_code != 200:
        print("❌ 失敗：")
        print(resp.text[:500])
        return

    data = resp.json()

    # 看最外層 keys
    print("\n🔍 最外層 keys：", list(data.keys()))

    # 直接看 data['records'] 這個 dict 裡面有什麼
    records_top = data.get("records")
    print("\n🔍 data['records'] 型別：", type(records_top))

    if isinstance(records_top, dict):
        print("🔍 data['records'] keys：", list(records_top.keys()))

        # 逐一檢查裡面的每個 key 是什麼型別
        for k, v in records_top.items():
            print(f"   - key = {k!r}，型別 = {type(v)}")
            # 如果是 list，就顯示長度跟第一筆
            if isinstance(v, list):
                print(f"     -> 這是 list，長度 = {len(v)}")
                if len(v) > 0:
                    print("     -> 第一筆內容示範：")
                    print(json.dumps(v[0], ensure_ascii=False, indent=2))

    print("\n🧾 原始 JSON 前 300 字：")
    print(resp.text[:300])


debug_cwa_records()


🚀 呼叫中央氣象署 API ...
HTTP 狀態碼： 200

🔍 最外層 keys： ['success', 'result', 'records']

🔍 data['records'] 型別： <class 'dict'>
🔍 data['records'] keys： ['Station']
   - key = 'Station'，型別 = <class 'list'>
     -> 這是 list，長度 = 1264
     -> 第一筆內容示範：
{
  "StationName": "九份二山",
  "StationId": "C1I230",
  "Maintainer": "中央氣象署",
  "ObsTime": {
    "DateTime": "2025-11-26T22:50:00+08:00"
  },
  "GeoInfo": {
    "Coordinates": [
      {
        "CoordinateName": "TWD67",
        "CoordinateFormat": "decimal degrees",
        "StationLatitude": "23.963714",
        "StationLongitude": "120.836856"
      },
      {
        "CoordinateName": "WGS84",
        "CoordinateFormat": "decimal degrees",
        "StationLatitude": "23.962025",
        "StationLongitude": "120.845272"
      }
    ],
    "StationAltitude": "837.0",
    "CountyName": "南投縣",
    "TownName": "國姓鄉",
    "CountyCode": "10008",
    "TownCode": "10008100"
  },
  "RainfallElement": {
    "Now": {
      "Precipitation": "0.0"
    },
    "Past1

cwa_rain_step2_to_excel

CWA-1F2E0711-4AD0-4F44-9344-E3799E4D8950

In [8]:
# 中央氣象署雨量站 → 篩選指定測站 → 存成 Excel
# 步驟：
# 1. 呼叫 O-A0002-001 API
# 2. 從 data["records"]["Station"] 拿出全部測站
# 3. 只保留我們指定的那幾個 StationId
# 4. 整理欄位，存成 xlsx

import requests
import pandas as pd
from datetime import datetime

# ===== 把這裡換成你的授權碼 =====
API_KEY = "CWA-1F2E0711-4AD0-4F44-9344-E3799E4D8950"  # 換成你的 key

# ===== CWA 自動雨量站 API =====
API_URL = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0002-001"

# ===== 你需要的測站清單（所有場次共同擁有） =====
TARGET_STATIONS = [
    "01J100",
    "01J930",
    "01J970",
    "01K060",
    "01L360",
    "01L390",
    "01L480",
    "01L490",
    "01L910",
    "01M010",
    "C0K330",
]


def fetch_cwa_rain_json(api_url, api_key):
    """向中央氣象署雨量 API 發送請求，回傳 JSON (dict)。"""
    params = {
        "Authorization": api_key
    }
    response = requests.get(api_url, params=params, timeout=20)
    response.raise_for_status()
    return response.json()


def pick_wgs84_coordinate(geo_info):
    """
    從 GeoInfo 裡面挑出 WGS84 的經緯度。
    結構大概是：
    "GeoInfo": {
        "Coordinates": [
            { "CoordinateName": "TWD67", ... },
            { "CoordinateName": "WGS84", ... }
        ],
        "StationAltitude": "837.0",
        ...
    }
    """
    if geo_info is None:
        return None, None, None, None, None

    coords = geo_info.get("Coordinates", [])
    county_name = geo_info.get("CountyName")
    town_name = geo_info.get("TownName")
    altitude_str = geo_info.get("StationAltitude")

    wgs_lat = None
    wgs_lon = None

    # 先找 CoordinateName == "WGS84"
    for c in coords:
        if c.get("CoordinateName") == "WGS84":
            wgs_lat = c.get("StationLatitude")
            wgs_lon = c.get("StationLongitude")
            break

    # 如果沒找到 WGS84，就退而求其次拿第一個
    if wgs_lat is None and len(coords) > 0:
        first = coords[0]
        wgs_lat = first.get("StationLatitude")
        wgs_lon = first.get("StationLongitude")

    # 高度轉成 float（可以失敗就 None）
    altitude = None
    try:
        if altitude_str is not None:
            altitude = float(altitude_str)
    except ValueError:
        altitude = None

    return wgs_lat, wgs_lon, altitude, county_name, town_name


def safe_precip(rainfall_element, key_name):
    """
    從 RainfallElement 取出各時間的 Precipitation。
    範例結構：
    "RainfallElement": {
        "Now": {"Precipitation": "0.0"},
        "Past10Min": {"Precipitation": "0.0"},
        ...
    }
    """
    if rainfall_element is None:
        return None

    block = rainfall_element.get(key_name)
    if block is None:
        return None

    value_str = block.get("Precipitation")
    try:
        if value_str is not None:
            return float(value_str)
    except ValueError:
        return None
    return None


def build_dataframe_for_target_stations(json_data, target_stations):
    """
    從 CWA JSON 中：
    - 取出 data["records"]["Station"]
    - 只保留指定 target_stations
    - 整理成 pandas DataFrame
    """
    records = json_data.get("records", {})
    stations = records.get("Station", [])

    rows = []
    found_ids = set()

    for st in stations:
        station_id = st.get("StationId")

        if station_id not in target_stations:
            continue  # 不在清單內就跳過

        found_ids.add(station_id)

        station_name = st.get("StationName")
        maintainer = st.get("Maintainer")

        # 觀測時間
        obs_time_info = st.get("ObsTime", {})
        obs_time = obs_time_info.get("DateTime")

        # 地理資訊
        geo_info = st.get("GeoInfo", {})
        lat, lon, altitude, county_name, town_name = pick_wgs84_coordinate(geo_info)

        # 雨量資訊
        rainfall = st.get("RainfallElement", {})

        now_10min = safe_precip(rainfall, "Now")
        past_10min = safe_precip(rainfall, "Past10Min")
        past_1hr = safe_precip(rainfall, "Past1hr")
        past_3hr = safe_precip(rainfall, "Past3hr")
        past_6hr = safe_precip(rainfall, "Past6Hr")
        past_12hr = safe_precip(rainfall, "Past12hr")
        past_24hr = safe_precip(rainfall, "Past24hr")
        past_2days = safe_precip(rainfall, "Past2days")
        past_3days = safe_precip(rainfall, "Past3days")

        row = {
            "StationId": station_id,
            "StationName": station_name,
            "Maintainer": maintainer,
            "CountyName": county_name,
            "TownName": town_name,
            "Latitude_WGS84": lat,
            "Longitude_WGS84": lon,
            "Altitude_m": altitude,
            "ObsTime": obs_time,
            "Rain_Now_10min": now_10min,
            "Rain_Past10Min": past_10min,
            "Rain_Past1hr": past_1hr,
            "Rain_Past3hr": past_3hr,
            "Rain_Past6Hr": past_6hr,
            "Rain_Past12hr": past_12hr,
            "Rain_Past24hr": past_24hr,
            "Rain_Past2days": past_2days,
            "Rain_Past3days": past_3days,
        }

        rows.append(row)

    df = pd.DataFrame(rows)

    # 額外檢查有哪些 station 沒被抓到（可能代碼不在這個資料集）
    missing = [sid for sid in target_stations if sid not in found_ids]
    if len(missing) > 0:
        print("⚠️ 下列測站在此 API 回傳資料中找不到：", missing)

    # 固定欄位順序（如果欄位存在就照順序排）
    col_order = [
        "ObsTime",
        "StationId",
        "StationName",
        "CountyName",
        "TownName",
        "Latitude_WGS84",
        "Longitude_WGS84",
        "Altitude_m",
        "Maintainer",
        "Rain_Now_10min",
        "Rain_Past10Min",
        "Rain_Past1hr",
        "Rain_Past3hr",
        "Rain_Past6Hr",
        "Rain_Past12hr",
        "Rain_Past24hr",
        "Rain_Past2days",
        "Rain_Past3days",
    ]
    existing_cols = [c for c in col_order if c in df.columns]
    other_cols = [c for c in df.columns if c not in existing_cols]
    df = df[existing_cols + other_cols]

    return df


def save_df_to_excel(df, base_filename="CWA_rain_targets"):
    """將 DataFrame 存成 xlsx 檔案，檔名帶時間戳記。"""
    ts = datetime.now().strftime("%Y%m%d_%H%M")
    filename = f"{base_filename}_{ts}.xlsx"
    df.to_excel(filename, index=False)
    return filename


def run_download_targets_to_excel():
    """
    1. 抓 CWA JSON
    2. 只留指定測站
    3. 存成 xlsx
    """
    json_data = fetch_cwa_rain_json(API_URL, API_KEY)
    df = build_dataframe_for_target_stations(json_data, TARGET_STATIONS)
    print("✅ 篩選後測站筆數：", len(df))

    filename = save_df_to_excel(df)
    print("✅ 已輸出檔案：", filename)


# 直接執行一次
run_download_targets_to_excel()


✅ 篩選後測站筆數： 11
✅ 已輸出檔案： CWA_rain_targets_20251126_2323.xlsx


整理 IP01~IP28 的颱風分頁與雨量站欄位

In [10]:
import os
import pandas as pd

# ===== 路徑設定 =====
INPUT_DIR = "./old_data"   # 來源資料夾
OUTPUT_DIR = "./DATA"      # 輸出資料夾
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ===== 要保留的分頁（颱風場次）=====
KEEP_SHEETS = [
    "2001_桃芝",
    "2004_敏督利",
    "2005_海棠",
    "2008_辛樂克",
    "2009_莫拉克",
    "2012_蘇拉",
    "2013_蘇力",
    "2015_蘇迪勒",
    "2016_梅姬",
    "2017_海棠",
]

# ===== 固定順序的 11 個雨量站欄位 =====
RAIN_STATIONS = [
    "C0K330",
    "01J100",
    "01J930",
    "01J970",
    "01K060",
    "01L360",
    "01L390",
    "01L480",
    "01L490",
    "01L910",
    "01M010",
]

DEPTH_COL = "Depth (cm)"  # 目標欄位名稱（最後一欄）


def process_one_ip_file(ip_index):
    """處理單一 IPxx.xlsx 檔案。"""
    filename = f"IP{ip_index:02d}.xlsx"
    input_path = os.path.join(INPUT_DIR, filename)

    if not os.path.exists(input_path):
        print(f"⚠️  檔案不存在，略過：{input_path}")
        return

    print(f"\n📂 處理檔案：{input_path}")
    xls = pd.ExcelFile(input_path)
    sheet_names = xls.sheet_names
    print("   原始分頁：", sheet_names)

    output_path = os.path.join(OUTPUT_DIR, filename)
    writer = pd.ExcelWriter(output_path, engine="openpyxl")

    for sheet in KEEP_SHEETS:
        if sheet not in sheet_names:
            print(f"   ⚠️ 分頁「{sheet}」不存在，略過。")
            continue

        df = pd.read_excel(input_path, sheet_name=sheet)

        # 檢查 Depth 欄位
        if DEPTH_COL not in df.columns:
            print(f"   ❌ 分頁「{sheet}」缺少欄位：{DEPTH_COL}，此分頁略過。")
            continue

        # 確保 11 個雨量站欄位都存在，若缺就建立全為 NaN 的欄位（避免程式直接炸掉）
        for st in RAIN_STATIONS:
            if st not in df.columns:
                print(f"   ⚠️ 分頁「{sheet}」缺少雨量站欄位：{st}，將建立空欄位（NaN）。")
                df[st] = pd.NA

        # 依固定順序選出 11 站 + Depth(cm)
        ordered_cols = RAIN_STATIONS + [DEPTH_COL]
        df_out = df[ordered_cols]

        # 寫入新檔的同名分頁
        df_out.to_excel(writer, sheet_name=sheet, index=False)
        print(f"   ✅ 已寫入分頁「{sheet}」，欄位：{ordered_cols}")

    writer.close()
    print(f"💾 已輸出整理後檔案：{output_path}")


def main():
    # IP01 ~ IP28，全數嘗試，有的就處理，沒有就略過
    for ip in range(1, 29):
        process_one_ip_file(ip)


if __name__ == "__main__":
    main()



📂 處理檔案：./old_data\IP01.xlsx
   原始分頁： ['2001_桃芝', '2004_敏督利', '2005_海棠', '2008_辛樂克', '2009_莫拉克', '2012_蘇拉', '2013_蘇力', '2015_蘇迪勒', '2016_梅姬', '2017_尼莎', '2017_海棠', '2021_盧碧', '2023_杜蘇芮']
   ✅ 已寫入分頁「2001_桃芝」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010', 'Depth (cm)']
   ✅ 已寫入分頁「2004_敏督利」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010', 'Depth (cm)']
   ✅ 已寫入分頁「2005_海棠」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010', 'Depth (cm)']
   ✅ 已寫入分頁「2008_辛樂克」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010', 'Depth (cm)']
   ✅ 已寫入分頁「2009_莫拉克」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010', 'Depth (cm)']
   ✅ 已寫入分頁「2012_蘇拉」，欄位：['C0K330', '01J100', '01J930', '01J970', '01K06

「沒淹水改成 -1」用程式

In [15]:
from adjust_depth_no_flood import run_adjust_all_ip
run_adjust_all_ip()



📂 處理 IP01.xlsx ...
   ▶ 2001_桃芝：有淹水，保留原始 Depth
   ▶ 2004_敏督利：有淹水，保留原始 Depth
   ▶ 2005_海棠：有淹水，保留原始 Depth
   ▶ 2008_辛樂克：有淹水，保留原始 Depth
   ▶ 2009_莫拉克：有淹水，保留原始 Depth
   ▶ 2012_蘇拉：有淹水，保留原始 Depth
   ▶ 2013_蘇力：有淹水，保留原始 Depth
   ▶ 2015_蘇迪勒：有淹水，保留原始 Depth
   ▶ 2016_梅姬：有淹水，保留原始 Depth
   ▶ 2017_海棠：有淹水，保留原始 Depth
   💾 已輸出： ./adjusted_DATA\IP01.xlsx

📂 處理 IP02.xlsx ...
   ▶ 2001_桃芝：有淹水，保留原始 Depth
   ▶ 2004_敏督利：有淹水，保留原始 Depth
   ▶ 2005_海棠：有淹水，保留原始 Depth
   ▶ 2008_辛樂克：沒淹水，將 Depth 改成 -1
   ▶ 2009_莫拉克：有淹水，保留原始 Depth
   ▶ 2012_蘇拉：有淹水，保留原始 Depth
   ▶ 2013_蘇力：有淹水，保留原始 Depth
   ▶ 2015_蘇迪勒：有淹水，保留原始 Depth
   ▶ 2016_梅姬：有淹水，保留原始 Depth
   ▶ 2017_海棠：沒淹水，將 Depth 改成 -1
   💾 已輸出： ./adjusted_DATA\IP02.xlsx

📂 處理 IP03.xlsx ...
   ▶ 2001_桃芝：有淹水，保留原始 Depth
   ▶ 2004_敏督利：有淹水，保留原始 Depth
   ▶ 2005_海棠：沒淹水，將 Depth 改成 -1
   ▶ 2008_辛樂克：沒淹水，將 Depth 改成 -1
   ▶ 2009_莫拉克：有淹水，保留原始 Depth
   ▶ 2012_蘇拉：有淹水，保留原始 Depth
   ▶ 2013_蘇力：沒淹水，將 Depth 改成 -1
   ▶ 2015_蘇迪勒：沒淹水，將 Depth 改成 -1
   ▶ 2016_梅姬：沒淹水，將 Depth 改成 -1
   ▶ 2017_海棠：沒淹水，將 Dept

即時雨量抓取並輸出成模型輸入格式.py

CWA-1F2E0711-4AD0-4F44-9344-E3799E4D8950

In [ ]:
from realtime_rain_fetcher import realtime_rain_window

df = realtime_rain_window(
    api_key="CWA-1F2E0711-4AD0-4F44-9344-E3799E4D8950",
    output_path="./realtime_rain_input.xlsx",
    rain_key="Past10Min",   # 或 "Past1hr" / "Past3hr" 等
)

print(df.tail(3))  # 看看最後幾列


"""
(版本: 3.10.7)
####
api_key 需要自行到中央氣象署網站申請
--根據下面的提示，跟新rain_key並依照設定的時間重複執行這段程式碼，就可以獲得xlsx (程式會自動增加即控制行數，只要重複執行即可)。
(xlsx即為AI模式的即時資料，意即透過xlsx的時間數據，讓AI計算未來四小時之淹水深度)
####


===========================================================
RainKey（rain_key 參數）說明

中央氣象署自動雨量站 API（O-A0002-001）中，
每個測站都有一個 RainfallElement 區塊，裡面包含
多種雨量時間尺度的欄位。

你可以自由選擇要抓哪一種雨量欄位，只要把
rain_key="XXXXX" 這個字串改掉即可。

以下為常見可用欄位（實際欄位會依 API 回傳為準）：

-----------------------------------------------------------
可選 RainKey 及其意義：

"Now"
    ➤ 當下雨量（立即值）

"Past10Min"
    ➤ 過去 10 分鐘累積雨量（常用）

"Past1hr"
    ➤ 過去 1 小時累積雨量（模型常用）

"Past3hr"
    ➤ 過去 3 小時累積雨量

"Past6hr"
    ➤ 過去 6 小時累積雨量

"Past12hr"
    ➤ 過去 12 小時累積雨量

"Past24hr"
    ➤ 過去 24 小時累積雨量

"Daily"
    ➤ 今日累積雨量（每日 00:00 開始計算）

-----------------------------------------------------------
⚠ 注意事項：

1. 你要確保模型訓練時使用的特徵跟這裡一致。
   例如模型用 3 小時雨量訓練，那 rain_key 就要用 "Past3hr"。

2. 若 API 沒回傳某個欄位，程式會自動填 0.0。

3. 所有測站欄位順序固定由 TARGET_STATIONS 控制，
   不會因為 API 回傳順序不同而亂掉。

===========================================================
"""



📈 追加一列，即時雨量資料已更新，目前總列數：100（最多 100 列）
✅ 已儲存即時雨量時間序列檔案： ./realtime_rain_input.xlsx
   欄位順序： ['C0K330', '01J100', '01J930', '01J970', '01K060', '01L360', '01L390', '01L480', '01L490', '01L910', '01M010']
   使用雨量欄位 RainfallElement['Past10Min']
    C0K330  01J100  01J930  01J970  01K060  01L360  01L390  01L480  01L490  \
97     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
98     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
99     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    01L910  01M010  
97     0.0     0.0  
98     0.0     0.0  
99     0.0     0.0  


'\napi_key 需要自行到中央氣象署網站申請\n--根據下面的提示，跟新rain_key並依照設定的時間重複執行這段程式碼，就可以獲得xlsx。\n===========================================================\nRainKey（rain_key 參數）說明\n\n中央氣象署自動雨量站 API（O-A0002-001）中，\n每個測站都有一個 RainfallElement 區塊，裡面包含\n多種雨量時間尺度的欄位。\n\n你可以自由選擇要抓哪一種雨量欄位，只要把\nrain_key="XXXXX" 這個字串改掉即可。\n\n以下為常見可用欄位（實際欄位會依 API 回傳為準）：\n\n-----------------------------------------------------------\n可選 RainKey 及其意義：\n\n"Now"\n    ➤ 當下雨量（立即值）\n\n"Past10Min"\n    ➤ 過去 10 分鐘累積雨量（常用）\n\n"Past1hr"\n    ➤ 過去 1 小時累積雨量（模型常用）\n\n"Past3hr"\n    ➤ 過去 3 小時累積雨量\n\n"Past6hr"\n    ➤ 過去 6 小時累積雨量\n\n"Past12hr"\n    ➤ 過去 12 小時累積雨量\n\n"Past24hr"\n    ➤ 過去 24 小時累積雨量\n\n"Daily"\n    ➤ 今日累積雨量（每日 00:00 開始計算）\n\n-----------------------------------------------------------\n⚠ 注意事項：\n\n1. 你要確保模型訓練時使用的特徵跟這裡一致。\n   例如模型用 3 小時雨量訓練，那 rain_key 就要用 "Past3hr"。\n\n2. 若 API 沒回傳某個欄位，程式會自動填 0.0。\n\n3. 所有測站欄位順序固定由 TARGET_STATIONS 控制，\n   不會因為 API 回傳順序不同而亂掉。\n\n===========================================================\n'